# Credmark Modeling Framework Example for Jupyter notebook
## Account - Tokens

version: 2022.6.23

In [ ]:
from credmark.cmf.engine.model_loader import ModelLoader
from credmark.cmf.engine.context import EngineModelContext

from credmark.dto import *
from credmark.cmf.types import *

from credmark.cmf.types.ledger import (
    BlockTable, ContractTable,
    LogTable, ReceiptTable, TokenTable, TokenTransferTable,
    TraceTable, TransactionTable, LedgerTable,
    LedgerAggregate, LedgerModelOutput
)

from credmark.cmf.engine.dev_models.console import get_dt, get_block, log_output

## Initialize

<div class="alert alert-block alert-info">
    <b>Note:</b> Change to a web3 provider you have in `params` below.
</div>

In [ ]:
model_loader = ModelLoader(['../models'], None, True)

params = {'chain_id': 1,
          'block_number': None,
          'model_loader': model_loader,
          'chain_to_provider_url': {'1': 'http://192.168.68.122:10444'},
          'api_url': None,
          'run_id': None,
          'console': True,
          'use_local_models': '*' # '*' # 'token.price'
         }
context = EngineModelContext.create_context(**params)

ledger = context.ledger
run_model = context.run_model
models = context.models
block_number = context.block_number
chain_id = context.chain_id
web3 = context.web3
run_model_historical = context.historical.run_model_historical
run_model_historical_blocks = context.historical.run_model_historical_blocks

## Convex Token

In [ ]:
convex_token = Token(address='0x4e3FBD56CD56c3e72c1403e103b45Db9da5B9D2B')
df_convex = (context.ledger.get_erc20_transfers(
    columns=[TokenTransferTable.Columns.BLOCK_NUMBER,
             TokenTransferTable.Columns.LOG_INDEX,
             TokenTransferTable.Columns.TO_ADDRESS,
             TokenTransferTable.Columns.FROM_ADDRESS,
             TokenTransferTable.Columns.TOKEN_ADDRESS],
    where=(f'{TokenTransferTable.Columns.TOKEN_ADDRESS} = lower(\'{convex_token.address}\')'),
).to_dataframe().sort_values(['block_number', 'log_index']))
df_convex

## Account

In [ ]:
acc_1 = Account(address='0x5291fBB0ee9F51225f0928Ff6a83108c86327636')

### Tokens that have been owned by the account

In [ ]:
context.ledger.get_erc20_transfers(
    columns=[TokenTransferTable.Columns.TOKEN_ADDRESS],
    where=' '.join(
        [f"{TokenTransferTable.Columns.FROM_ADDRESS}='{acc_1.address}'",
         "or",
         f"{TokenTransferTable.Columns.TO_ADDRESS}='{acc_1.address}'"]),
    group_by=TokenTransferTable.Columns.TOKEN_ADDRESS
).to_dataframe()

### Token Transfers for this account

In [ ]:
group_cols = [TokenTransferTable.Columns.BLOCK_NUMBER,
             TokenTransferTable.Columns.LOG_INDEX,
             TokenTransferTable.Columns.TO_ADDRESS,
             TokenTransferTable.Columns.FROM_ADDRESS,
             TokenTransferTable.Columns.TRANSACTION_HASH,
             TokenTransferTable.Columns.TOKEN_ADDRESS]

df_tt = (context.ledger.get_erc20_transfers(
    columns=group_cols,
    aggregates=[context.ledger.Aggregate(f'sum(case when {TokenTransferTable.Columns.TO_ADDRESS} = lower(\'{acc_1.address}\') THEN {TokenTransferTable.Columns.VALUE} else -{TokenTransferTable.Columns.VALUE} END)', 'sum_value')],
    where=(f'{TokenTransferTable.Columns.TO_ADDRESS} = lower(\'{acc_1.address}\') or '
           f'{TokenTransferTable.Columns.FROM_ADDRESS} = lower(\'{acc_1.address}\')'),
    group_by=','.join(group_cols))
     .to_dataframe().sort_values(['block_number', 'log_index']).reset_index())

df_tt

### The Tokens

In [ ]:
wBTC = Token(address='0x2260fac5e5542a773aa44fbcfedf7c193bc2c599')
wBTC.scaled(748139023)

renBTC = Token(address='0xeb4c2781e4eba804ce9a9803c67d0893436bb27d')
renBTC

t1 = Token(address='0xdac17f958d2ee523a2206206994597c13d831ec7')
t2 = Token(address='0x2260fac5e5542a773aa44fbcfedf7c193bc2c599')
t3 = Token(address='0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2')

In [ ]:
curve_usd_btc_eth = Token(address='0xc4ad29ba4b3c580e6d59105fff484999997675ff')
print(curve_usd_btc_eth.balance_of(acc_1.address))

ve_crv = Token(address='0x5f3b5DfEb7B28CDbD7FAba78963EE202a494e2A2')
print(ve_crv.balance_of(acc_1.address))
print(ve_crv.functions.locked(acc_1.address).call())

convex_voter_proxy = Contract(address='0x989AEb4d175e16225E39E87d0D97A3360524AD80')
convex_voter_proxy.abi.functions

In [ ]:
from decimal import Decimal
pos_dict = {}
for n, r in df_tt.iterrows():
    tok_address = r['token_address']
    value = r['sum_value']
    pos_dict[tok_address] = pos_dict.get(tok_address, Decimal(0)) + Decimal(value)
    if pos_dict[tok_address] == 0:
        tok = Token(address=tok_address)
        print(tok.symbol, tok_address)
        del pos_dict[tok_address]

display(pos_dict)

token_dict = {}
for tok_address, amount in pos_dict.items():
    tok = Token(address=tok_address)
    token_dict[tok_address] = tok
    pos_dict[tok_address] = tok.scaled(amount)

pos_dict

### Curve Pool

In [ ]:
pool_info = context.models.curve_fi.pool_info(address='0xc4ad29ba4b3c580e6d59105fff484999997675ff')
pool_info = context.models.curve_fi.pool_info(address=pool_info['address'])
pool_info

In [ ]:
pool = Contract(address='0xd51a44d3fae010294c616388b506acda1bfaae46')
b1,b2,b3=(pool.functions.balances(0).call(),
          pool.functions.balances(1).call(),
          pool.functions.balances(2).call())

In [ ]:
cc = Contract(address=pool_info['gauges'][0])
print(cc.functions.balanceOf(acc_1.address.checksum).call())
print(cc.functions.working_balances(acc_1.address.checksum).call())
print(cc.functions.claimable_tokens(acc_1.address.checksum).call())